<p align ="center">
AME 5763
</p>
<p align ="center">
Homework 5
</p>
<p align="center">
Blake T Johnson
</p>
<p align="center">
November 13, 2024
</p>

In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from mpl_toolkits.mplot3d import Axes3D
import sympy as sp
from IPython.core.display import HTML
from sympy import Matrix
import matplotlib.tri as tri

import sys

sys.path.append('/Users/blakejohnson/Documents/Fall 2024/Finite Elements')
from finite_element_equations import *

sys.path.append('/Users/blakejohnson/Documents/Fall 2024/Finite Elements')
from finite_elements import *

# Problem 8.1
Consider a problem on a rectangular $ (2 m \times 1 m)$ domain as shown in Figure 8.18. The conductivity is $k = 4 W^{\circ} C^{-1}$. $T = 10^{\circ} C$ is prescribed along the edge CD. Edges AB and AD are insulated, i.e. $\bar{q} = 0 W m^{-1}$ ; along the edge DC, the boundary flux is $\bar{q} = 30 W m^{-1}$ . A constant heat source is given: $s = 50 Wm^{-2}$.
Find the nodal temperature and nodal fluxes; evaluate the element matrices by Gauss quadrature. Use a single rectangular finite element with node numbering shown in Figure 8.19 so that the local and global node numberings coincide.

![figure_1](figure_1.png)


![Image Description](figure_2.png)


Note: The text of the problem states that T = 10 is prescribed along CD. However the image in the figure shows T = 10 appears to be along BC. It also says that $\bar{q} = 30$ is along DC. This makes me think that there is a typo and that T = 10 is along BC, so I calculated the values based on the parameters of the image, not the text.

In [2]:
'''
Set up our given variables
'''
# Define the symbols for xi and eta globally
xi, eta = sp.symbols('xi eta')

k = 4
T = 10
q_0 = 0
q_1 = 30  # Flux value at the boundary
s = 50

In [3]:
'''
Step 1: The Element Matrix
'''
finite_element_preprocessor("quad",[(2,0),(2,1),(0,1),(0,0)])


Coordinate Matrix:
Node 1: Global (2, 0), Parent (-1, -1)
Node 2: Global (2, 1), Parent (1, -1)
Node 3: Global (0, 1), Parent (1, 1)
Node 4: Global (0, 0), Parent (-1, 1)

Shape Functions (N):
⎡(0.25 - 0.25⋅ξ)⋅(1 - η)⎤
⎢                       ⎥
⎢(1 - η)⋅(0.25⋅ξ + 0.25)⎥
⎢                       ⎥
⎢(η + 1)⋅(0.25⋅ξ + 0.25)⎥
⎢                       ⎥
⎣(0.25 - 0.25⋅ξ)⋅(η + 1)⎦
Gradient of N with respect to xi and eta:
⎡0.25⋅η - 0.25  0.25 - 0.25⋅η   0.25⋅η + 0.25  -0.25⋅η - 0.25⎤
⎢                                                            ⎥
⎣0.25⋅ξ - 0.25  -0.25⋅ξ - 0.25  0.25⋅ξ + 0.25  0.25 - 0.25⋅ξ ⎦

Coordinate Matrix (Global Coordinates):
⎡2  0⎤
⎢    ⎥
⎢2  1⎥
⎢    ⎥
⎢0  1⎥
⎢    ⎥
⎣0  0⎦

Jacobian Matrix (J):
⎡ 0    0.5⎤
⎢         ⎥
⎣-1.0   0 ⎦

Transpose of the Jacobian (J^T):
⎡ 0   -1.0⎤
⎢         ⎥
⎣0.5   0  ⎦

Jacobian Determinant (|J|):
0.50000

Jacobian Inverse (J^-1):
⎡ 0   -1.0⎤
⎢         ⎥
⎣2.0   0  ⎦

B Matrix:
⎡0.25 - 0.25⋅ξ  0.25⋅ξ + 0.25  -0.25⋅ξ - 0.25  0.25⋅ξ - 0.25⎤
⎢  

In [4]:
B = B_matrix(Grad_N, J_inv)
print("\n B Transpose")
sp.pprint(B)


NameError: name 'Grad_N' is not defined

In [ ]:
'''
Step 6: The Conduction Matrix
'''

K = stiffness_matrix_kBBJ(k, B, det_J, (-1, 1), (-1, 1))

In [ ]:
'''
Calculate the Source Matrix
'''
f_Omega = f_Omega_Ns(N,s,det_J, (-1,1),(-1,1))



In [ ]:
'''
Step 8: The Boundary Flux Matrix (f_gamma)
'''
f_gamma = f_Gamma_q(N,q_1,'xi',1,(1,-1))

In [ ]:
'''
Step 9: RHS Matrix
'''
RHS = rhs(f_Omega, f_gamma)
RHS = RHS.T

In [ ]:
dof = dofs([[4,1,2,3]])
print(dof)

In [ ]:
K_global, RHS_global = global_stiffness(k,K,RHS,boundary_conditions=(-1,1))

In [ ]:
'''
Step 14: Solve for Nodal Temperatures
'''

# Solve the matrix equation to get the nodal temperatures
# I know that node 1 and 2 are 10°C so I can solve for the other nodes
nodal_temperatures = K_modified_matrix.LUsolve(RHS_modified_matrix)
print("\nNodal Temperatures:")
print(nodal_temperatures)

In [ ]:
'''
Step 15: Calculate Nodal Fluxes
'''
# The nodal fluxes can be determined by multiplying the original K matrix by the nodal temperatures
nodal_fluxes = K_global_numeric * nodal_temperatures - RHS
print("\nNodal Fluxes:")
print(nodal_fluxes)

# Problem 8.3
A finite element mesh consisting of a rectangular and a triangular element is shown in Figure 8.21. The dimensions of the plate are in meters. A constant temperature$ $T = 10^\circ $, C  is prescribed along the boundary $ y = 0 $. A constant and linear boundary flux, as shown in Figure 8.21, is applied along the edges $ y = x + 2 $ and $ x = 0 $, respectively. The edge $ x = 2 $ is insulated. A point source $ P = 10 \, W $ is applied at $ (0, 2) \, m $. The material is isotropic with $ k = 1 , W/^\circ C $ for element 1 and $k = 2 \, W/^\circ C \$ for element 2. Compute the nodal temperatures and fluxes at the center points of the two elements.


![figure_3](figure_3.png)

In [ ]:
'''
Calculate Element 1 (Rectangular Element) Stiffness Matrix
'''

In [ ]:
'''
Step 1: The Element Matrix
'''

# Define coordinates of each node in the element
E1 = sp.Matrix([[0, 0],  # Node 1
                [2, 0],  # Node 2
                [2, 2],  # Node 3
                [0, 2]]) # Node 5


In [ ]:
'''
Step 2: Find the Shape Functions
'''
# I decided to set up the shape funcitons first this time. I skipped that step in 8.1 and had to go back

# Define natural coordinates (xi, eta)
xi, eta = sp.symbols('xi eta')

# Define shape functions for a four-node rectangular element
N1 = (1/4) * (1 - xi) * (1 - eta)
N2 = (1/4) * (1 + xi) * (1 - eta)
N3 = (1/4) * (1 + xi) * (1 + eta)
N4 = (1/4) * (1 - xi) * (1 + eta)

# Collect shape functions into a matrix
N = sp.Matrix([N1, N2, N3, N4])

In [ ]:
'''
Step 3: The Gradient
'''

# Calculate partial derivatives of shape functions with respect to xi and eta
dN_dxi = [sp.diff(Ni, xi) for Ni in N]
dN_deta = [sp.diff(Ni, eta) for Ni in N]

# Set the matrix for the gradient
G = sp.Matrix([dN_dxi, dN_deta])

print("\nGradient Matrix G:")
print(G)


In [ ]:
'''
Step 4: The Jacobian
'''

# Multiply G and E1 to get the Jacobian matrix
J = G * E1

print("\nJacobian Matrix J:")
print(J)


In [ ]:
'''
Step 5: Determinant and Inverse of the Jacobian
'''

# Calculate the determinant of the Jacobian
det_J = J.det()

# Calculate the inverse of the Jacobian
J_inv = J.inv()

print("\nDeterminant of the Jacobian (det_J):")
print(det_J)

print("\nInverse of the Jacobian (J_inv):")
print(J_inv)


In [ ]:
'''
Step 6: Derivatives of the Shape Functions (B)
'''

# Calculate the B matrix (derivative of the shape functions with respect to global coordinates)
B = J_inv * G

print("\nB Matrix (Derivatives of Shape Functions in Global Coordinates):")
print(B)


In [ ]:
'''
Step 7: The Conduction Matrix
'''

# Define the thermal conductivity for Element 1
#This was given in the problem statement

k_1 = 1

# The material property matrix D for isotropic material (scalar k)
D = k_1

# Calculate the B^T * D * B product
BT_D_B = B.T * D * B

# Next I integrated over natural coordinates xi and eta, from -1 to 1
K_element1 = sp.integrate(sp.integrate(BT_D_B * det_J, (xi, -1, 1)), (eta, -1, 1))

print("\nConduction Matrix K for Element 1:")
print(K_element1)


In [ ]:
'''
Calculate Element 2 (Triangular Element) Stiffness Matrix
'''

In [ ]:
'''
Step 1: The Element Matrix
'''

# Define coordinates for triangular element (Element 2)
E2 = sp.Matrix([[2, 2],  # Node 3
                [2, 4],  # Node 4
                [0, 2]]) # Node 5



In [ ]:
'''
Step 2: Find the Shape Functions
'''
# Define shape functions for a three-node triangular element
# L1 = 1 - xi - eta, L2 = xi, L3 = eta
N_tri1 = 1 - xi - eta
N_tri2 = xi
N_tri3 = eta
N_tri = sp.Matrix([N_tri1, N_tri2, N_tri3])


In [ ]:
'''
Step 3: The Gradient
'''

# Calculate partial derivatives of triangular shape functions
dN_tri_dxi = [sp.diff(Ni, xi) for Ni in N_tri]
dN_tri_deta = [sp.diff(Ni, eta) for Ni in N_tri]

# Set the matrix for the gradient
G_tri = sp.Matrix([dN_tri_dxi, dN_tri_deta])


In [ ]:
'''
Step 4: The Jacobian
'''

# Multiply G_tri and E2 to get the Jacobian matrix for Element 2
J_tri = G_tri * E2


print("\nJacobian Matrix J:")
print(J)

In [ ]:
'''
Step 5: Determinant and Inverse of the Jacobian
'''

# Calculate the determinant of the Jacobian
det_J_tri = J_tri.det()


# Calculate the inverse of the Jacobian
J_inv_tri = J_tri.inv()

print("\nDeterminant of the Jacobian (det_J):")
print(det_J_tri)

print("\nInverse of the Jacobian (J_inv):")
print(J_inv_tri)

In [ ]:
'''
Step 6: Derivatives of the Shape Functions (B)
'''

# Calculate B matrix for triangular element in global coordinates
B_tri = J_inv_tri * G_tri

print("\nB Matrix (Derivatives of Shape Functions in Global Coordinates):")
print(B_tri)


In [ ]:
'''
Step 7: The Stiffness Matrix
'''

# Define the thermal conductivity for Element 2
k_2 = 2

# The material property matrix D for isotropic material (scalar k)
D = k_2

# Calculate the B^T * D * B product
BT_D_B_tri = B_tri.T * D * B_tri

# Now we integrate over natural coordinates xi and eta, from 0 to 1 (since it's a triangular element)
K_element2 = sp.integrate(sp.integrate(BT_D_B_tri * det_J_tri, (xi, 0, 1)), (eta, 0, 1))

print("\nStiffness Matrix K for Element 2:")
print(K_element2)


In [ ]:
'''
Global Stiffness Matrix
'''

In [ ]:
'''
Now I can set up the global stiffness matrix.
I have a total of 5 nodes in the mesh.
The global stiffness matrix will therefore be a 5x5 matrix.
I need to map the contributions from each element stiffness matrix to the 
corresponding rows and columns in the global stiffness matrix.
'''
# Number of nodes in the global system
num_nodes = 5

K_global = np.zeros((num_nodes, num_nodes))



# Element stiffness matrices (manually copied from earlier results)
K_element1 = np.array([[0.6667, -0.1667, -0.3333, -0.1667],
                       [-0.1667, 0.6667, -0.1667, -0.3333],
                       [-0.3333, -0.1667, 0.6667, -0.1667],
                       [-0.1667, -0.3333, -0.1667, 0.6667]])



K_element2 = np.array([[5.333, -1.333, -2.667, -1.333],
                       [-1.333, 5.333, -1.333, -2.667],
                       [-2.667, -1.333, 5.333, -1.333],
                       [-1.333, -2.667, -1.333, 5.333]])

'''
Element 1 (Rectangular Element) involves nodes 1, 2, 3, and 5.
I mapped the 4x4 element stiffness matrix for Element 1 into the global stiffness matrix based on the global node numbers:
- Node 1 maps to row/column 1.
- Node 2 maps to row/column 2.
- Node 3 maps to row/column 3.
- Node 5 maps to row/column 5.
'''
element1_nodes = [0, 1, 2, 4]  # Nodes 1, 2, 3, 5 (Python indexing: 0-based)

'''
Now, did the same for Element 2.
Element 2 (Triangular Element) involves nodes 3, 4, and 5.
I mapped the 3x3 element stiffness matrix for Element 2 into the global stiffness matrix based on the global node numbers:
- Node 3 maps to row/column 3.
- Node 4 maps to row/column 4.
- Node 5 maps to row/column 5.
'''

element2_nodes = [2, 3, 4]      # Nodes 3, 4, 5 (Python indexing: 0-based)

# Assemble Element 1 into the global stiffness matrix
for i in range(len(element1_nodes)):
    for j in range(len(element1_nodes)):
        K_global[element1_nodes[i], element1_nodes[j]] += K_element1[i, j]

# Assemble Element 2 into the global stiffness matrix
for i in range(len(element2_nodes)):
    for j in range(len(element2_nodes)):
        K_global[element2_nodes[i], element2_nodes[j]] += K_element2[i, j]

# Print the global stiffness matrix
print("\nGlobal Stiffness Matrix K_global:")
print(K_global)


In [ ]:
'''
Apply Boundary Conditions
'''

'''
Step 1: Define the Load Vector
I start by defining the load vector `F`, which will represent the applied loads on the system. 
Since I have 5 nodes in the mesh, the load vector will initially be set to zeros for all 5 nodes.
'''

F = np.zeros(num_nodes)

'''
Step 2: Apply Point Source at Node 5
There is a point source of 10 W applied at Node 5, so I assign this value to the corresponding entry in the load vector `F`.
Note: Node 5 is at index 4 in Python's 0-based indexing system.
'''

F[4] = 10

'''
Step 3: Apply Prescribed Temperatures at Nodes 1 and 2
The temperature is prescribed to be 10°C at nodes 1 and 2.
These nodes correspond to indices 0 and 1.
I will update the global stiffness matrix `K_global` and the load vector `F` to apply these boundary conditions.
'''

prescribed_nodes = [0, 1]
prescribed_temperature = 10

# Apply the boundary conditions for prescribed temperatures
for node in prescribed_nodes:
    '''
    Step 4: Modify the Global Stiffness Matrix for Prescribed Temperatures
    For each prescribed node, I set the entire row and column in the global stiffness matrix to zero.
    This represents the fact that these nodes have a fixed temperature, so there will be no contribution to other nodes.
    I then set the diagonal element to 1, which effectively enforces the temperature condition at these nodes.
    '''
    
    # Set corresponding row and column in K_global to zero
    K_global[node, :] = 0
    K_global[:, node] = 0
    
    # Set the diagonal element to 1
    K_global[node, node] = 1
    
    '''
    Step 5: Update the Load Vector for Prescribed Temperatures
    I set the corresponding entry in the load vector `F` to the prescribed temperature (10°C).
    This ensures that the solution will satisfy the temperature condition at these nodes.
    '''

    # Set the corresponding entry in the load vector to the prescribed temperature
    F[node] = prescribed_temperature

print("\nGlobal Stiffness Matrix K_global after applying boundary conditions:")
print(K_global)
print("\nLoad Vector F after applying boundary conditions:")
print(F)


In [ ]:
'''
Temperatures at Nodes
'''
# Solve for nodal temperatures
T = np.linalg.solve(K_global, F)

# Print the nodal temperatures
print("\nNodal Temperatures:")
print(T)

In [ ]:
'''
Calculate the Temperature and Heat Flux At the Center of Each Element
'''

# Define the nodal temperatures (using the values from the solved T array)
T_nodal = sp.Matrix([10, 10, T[2], T[3], T[4]])

In [ ]:
'''
For Element 1 (Rectangular Element)
'''

'''
Step 1: Define the Centroid Coordinates for Element 1 (Rectangular Element)
To calculate properties at the centroid of the rectangular element, I need to evaluate the relevant matrices at the center.
For a rectangular element in natural coordinates, the centroid is located at (xi, eta) = (0, 0).
'''

# Define center coordinates for rectangular element (xi = 0, eta = 0)
xi_value = 0
eta_value = 0

'''
Step 2: Substitute Centroid Coordinates into the B Matrix
The B matrix contains the derivatives of the shape functions with respect to global coordinates, which I use to calculate fluxes.
Here, I substitute the centroid coordinates (xi, eta) = (0, 0) into the B matrix to get the B matrix evaluated at the centroid.
'''

# Substitute center coordinates into the B matrix
B_centroid = B.subs({xi: xi_value, eta: eta_value})

'''
Step 3: Define the Nodal Temperatures for Element 1
Element 1 (the rectangular element) involves nodes 1, 2, 3, and 5.
To calculate the temperature and heat flux at the centroid, I need the temperature values at these nodes.
I use the solved nodal temperatures and arrange them into a column matrix.
'''

# Use nodal temperatures of Element 1 (nodes 1, 2, 3, 5)
T_element1 = sp.Matrix([T_nodal[0], T_nodal[1], T_nodal[2], T_nodal[4]])

'''
Step 4: Calculate the Temperature at the Centroid of Element 1
I use the shape functions to calculate the temperature at the centroid.
By evaluating the shape functions at (xi, eta) = (0, 0) and taking the dot product with the nodal temperatures,
I can determine the temperature at the centroid.
'''

# Calculate temperature at the centroid of Element 1 using shape functions
N_centroid = N.subs({xi: xi_value, eta: eta_value})
T_center_element1 = N_centroid.T * T_element1

print("\nTemperature at the centroid of Element 1:")
print(T_center_element1)

'''
Step 5: Calculate the Heat Flux at the Centroid of Element 1
The heat flux is calculated using Fourier's law: q = -k * B * T.
Here, I use the thermal conductivity `k_1`, the B matrix evaluated at the centroid, and the nodal temperature vector `T_element1`.
The resulting `q_element1_centroid` represents the heat flux at the centroid in global coordinates.
'''

# Calculate heat flux at the centroid of Element 1
q_element1_centroid = -k_1 * B_centroid * T_element1

print("\nHeat Flux at the centroid of Element 1 (in global coordinates):")
print(q_element1_centroid)


In [ ]:
'''
For Element 2 (Triangular Element)
'''

'''
Step 1: Define the Element Matrix for Element 2 (Triangular Element)
Element 2 is a triangular element with nodes at specific coordinates. These coordinates represent Node 3, Node 4, and Node 5.
'''

# Define coordinates for triangular element (Element 2)
E2 = sp.Matrix([[2, 2],  # Node 3
                [2, 4],  # Node 4
                [0, 2]]) # Node 5

'''
Step 2: Define Shape Functions for the Triangular Element
For a three-node triangular element, we use linear shape functions defined in natural coordinates (xi, eta).
The shape functions L1, L2, and L3 represent the contributions from each of the three nodes.
'''

# Define shape functions for a three-node triangular element
# L1 = 1 - xi - eta, L2 = xi, L3 = eta
N_tri1 = 1 - xi - eta
N_tri2 = xi
N_tri3 = eta
N_tri = sp.Matrix([N_tri1, N_tri2, N_tri3])

'''
Step 3: Calculate the Gradient of the Shape Functions
To determine how the shape functions change with respect to natural coordinates, I need to compute their partial derivatives.
This involves calculating the derivatives with respect to both xi and eta for each shape function.
'''

# Calculate partial derivatives of triangular shape functions
dN_tri_dxi = [sp.diff(Ni, xi) for Ni in N_tri]
dN_tri_deta = [sp.diff(Ni, eta) for Ni in N_tri]

# Define the gradient matrix G_tri for triangular element
G_tri = sp.Matrix([dN_tri_dxi, dN_tri_deta])

'''
Step 4: Compute the Jacobian Matrix for Element 2
The Jacobian matrix describes the transformation from natural coordinates (xi, eta) to global coordinates (x, y).
I multiply the gradient matrix G_tri by the element node coordinate matrix E2 to get the Jacobian matrix.
'''

# Multiply G_tri and E2 to get the Jacobian matrix for Element 2
J_tri = G_tri * E2
det_J_tri = J_tri.det()
J_inv_tri = J_tri.inv()

'''
Step 5: Derivatives of the Shape Functions in Global Coordinates (B Matrix)
To find the derivatives of the shape functions with respect to global coordinates, I use the inverse Jacobian.
The B matrix is obtained by multiplying the inverse Jacobian with the gradient matrix G_tri.
'''

# Calculate B matrix for triangular element in global coordinates
B_tri = J_inv_tri * G_tri

'''
Step 6: Evaluate B Matrix at the Centroid of the Triangular Element
The centroid of a triangular element in natural coordinates is located at (xi, eta) = (1/3, 1/3).
I substitute these values into the B matrix to evaluate it at the centroid.
'''

# Centroid of a triangular element in natural coordinates
xi_value = 1/3
eta_value = 1/3

# Substitute centroid coordinates into the B matrix for Element 2
B_centroid_tri = B_tri.subs({xi: xi_value, eta: eta_value})

'''
Step 7: Define the Nodal Temperatures for Element 2
Element 2 involves nodes 3, 4, and 5.
I use the nodal temperatures obtained from the solution to evaluate properties at the centroid of this element.
'''

# Use nodal temperatures of Element 2 (nodes 3, 4, 5)
T_element2 = sp.Matrix([T_nodal[2], T_nodal[3], T_nodal[4]])

'''
Step 8: Calculate the Temperature at the Centroid of Element 2
The temperature at the centroid is computed using the shape functions.
I evaluate the shape functions at the centroid coordinates and use them to interpolate the nodal temperatures.
'''

# Calculate temperature at the centroid of Element 2 using shape functions
N_centroid_tri = N_tri.subs({xi: xi_value, eta: eta_value})
T_center_element2 = N_centroid_tri.T * T_element2

print("\nTemperature at the centroid of Element 2:")
print(T_center_element2)

'''
Step 9: Calculate the Heat Flux at the Centroid of Element 2
Using Fourier's law, I calculate the heat flux at the centroid of the triangular element.
The heat flux is obtained by multiplying the negative thermal conductivity by the B matrix (evaluated at the centroid) and the nodal temperatures.
'''

# Calculate heat flux at the centroid of Element 2
k_2 = 2  # Thermal conductivity for Element 2
q_element2_centroid = -k_2 * B_centroid_tri * T_element2

print("\nHeat Flux at the centroid of Element 2 (in global coordinates):")
print(q_element2_centroid)


# Problem 8.4

Consider a triangular panel as shown in Figure 8.22. All dimensions are in meters. A constant temperature $T = 5^\circ C$ is prescribed along the boundary $y = 0$. A constant boundary flux $\bar{q} = 10 \, \text{W m}^{-1}$ is applied along the edges $x = 0.5$ and $y = x$. A constant heat source $s = 10 \, \text{W m}^{-2}$ is supplied over the panel, and a point source $P = 7 \, \text{W}$ acts at the origin. The material is isotropic with $k = 2 \, \text{W}^\circ \text{C}^{-1}$.

1. Number the nodes counterclockwise with nodes on the essential boundary numbered first. In this case, will the element matrices ($K^e$ and $f^e$) be any different from those of the global matrices?

2. Construct the conductance matrix.

3. Construct the boundary flux matrix resulting from the flux acting on the edges $x = 0.5$ and $y = x$.

4. Construct the source matrix consisting of uniformly distributed source $s = 10 \, \text{W m}^{-2}$ and point source $P = 7 \, \text{W}$.

5. Calculate the unknown temperature matrix.

6. Find the unknown reactions.

![figure_4](figure_4.png)

In [ ]:
'''
Construct the Conduction Matrix
'''

In [ ]:
'''
Step 1 Create the Element Matrix
'''
# Corrected element matrix to reflect the actual node positions and counterclockwise ordering
E = Matrix([[0, 0], [0.5, 0], [0.5, 0.5]])  # Node 1 (0,0), Node 2 (0.5, 0), Node 3 (0.5, 0.5)


In [ ]:
'''
 Step 2: Define the Shape Functions
'''

# Natural coordinates for the triangular element
xi, eta = sp.symbols('xi eta')

# Define shape functions for triangular element
N1 = 1 - xi - eta
N2 = xi
N3 = eta
N_tri = Matrix([N1, N2, N3])

In [ ]:
'''
Step 3: The Gradient
'''
# Calculate partial derivatives of triangular shape functions
partial_xi = [sp.diff(Ni, xi) for Ni in N_tri]
partial_eta = [sp.diff(Ni, eta) for Ni in N_tri]

# Define the gradient matrix G for the triangular element
G = Matrix([partial_xi, partial_eta])

In [ ]:
'''
Step 4: The Jacobian
'''

# Calculate the Jacobian matrix
J = G * E

'''
Step 5: Determinant and Inverse of the Jacobian
'''

# Calculate the determinant and inverse of the Jacobian
J_det = J.det()
J_inv = J.inv()

print("\nJacobian Matrix J:")
print(J)
print("\nDeterminant of Jacobian |J|:")
print(J_det)
print("\nInverse of Jacobian J^-1:")
print(J_inv)

In [ ]:
'''
Step 6: Derivatives of the Shape Functions (B)
'''

# Define material property: thermal conductivity
k = 2  # Thermal conductivity
D = k  # Isotropic material

B = J_inv * G

# Compute B^T * D * B and integrate over natural coordinates
# My results do not inclued xi and eta, which has me a little concerned
# However they canceled out in a previous problem, so I think that is what is happening here
B_T_D_B = B.T * D * B
K_element = sp.integrate(sp.integrate(B_T_D_B * J_det, (xi, 0, 1 - eta)), (eta, 0, 1))

print("\nB Matrix (Derivatives of Shape Functions in Global Coordinates):")
print(B)
print("\nConductance Matrix K for the Triangular Element:")
print(K_element)

In [ ]:
'''
Construct the boundary Flux Matrix resulting from the flux acting on the edges x = 0.5 and y = x
'''

# Flux (q) is applied along x = 0.5 and y = x
q = 10  # Flux value

# To calculate the flux matrix, substitute appropriate boundary values into shape functions
# Note: I wanted to integrate using -1 and 1 as my boundaries but I got weird results
# Since the element acts as the global coordinates, I used 0 and 0.5 as my boundaries
# This gave me a number that makes a bit more sense

N_boundary_x = N_tri.subs({xi: 0.5})
N_boundary_y = N_tri.subs({eta: xi})

# Integrate along the boundaries to calculate the flux matrix
f_gamma_x = sp.integrate(q * N_boundary_x.T, (eta, 0, 0.5))
f_gamma_y = sp.integrate(q * N_boundary_y.T, (xi, 0, 0.5))

# Add contributions from both boundaries
f_gamma = f_gamma_x + f_gamma_y

print("\nBoundary Flux Matrix f_gamma:")
print(f_gamma)

In [ ]:
''' 
Construct the Source Matrix consisting of uniformly distributed source s = 10
and the point source P = 7.
'''

# Given values s = 10 and P = 7 W
s = 10
P = 7

# Calculate the source matrix
f_Omega = sp.integrate(sp.integrate(s * N_tri.T * J_det, (xi, 0, 1 - eta)), (eta, 0, 1))


# Node 1 is located at the origin, so P = 7 W acts directly at Node 1
# I need to add the point source to the source matrix
# Convert the source matrix to a list for modification

f_Omega = f_Omega.tolist()  # Convert to list for modification
f_Omega[0][0] += P  # Correct indexing to properly add point source
f_Omega = Matrix(f_Omega)  # Convert back to Matrix

print("\nUpdated Source Matrix f_Omega:")
print(f_Omega)

In [ ]:
'''
Calculate the Unknown Temperature Matrix
'''

# Convert matrices to numerical form for calculations
K_element_numeric = np.array(K_element).astype(np.float64)
f_gamma_numeric = np.array(f_gamma).astype(np.float64).reshape(-1, 1)  # Ensure column vector shape
f_Omega_numeric = np.array(f_Omega).astype(np.float64).reshape(-1, 1)  

# Combine flux and source contributions
F_numeric = f_gamma_numeric + f_Omega_numeric  

# Known temperatures at nodes 1 and 2
T_known = 5  # Temperature at nodes A and B is 5°C

# Extract parts of K and F related to the unknown temperature at Node 3
# Since both Nodes 1 and 2 have known temperatures, we only need to solve for Node 3
K_unknown = K_element_numeric[2, 2]  # Extract the single entry for Node 3
F_unknown = F_numeric[2] - (K_element_numeric[2, 0] * T_known + K_element_numeric[2, 1] * T_known)

# Solve for the unknown temperature at Node 3
T_unknown = F_unknown / K_unknown

# Convert T_unknown from an array to a scalar value using .item()
T_C = T_unknown.item()

# Display the result
print("\nTemperature at Node 3: {:.2f} °C".format(T_C))

In [ ]:
'''
Calculate the Reaction Forces at Nodes 1 and 2
'''

# Form the full temperature vector
T_full = np.array([T_known, T_known, T_C]).reshape(-1, 1)

# Calculate the reaction forces using the global stiffness matrix
F_reaction = K_element_numeric @ T_full

print(f"The reaction matrisx is:\n{F_reaction}")
# Display the reaction forces at Nodes A and B
print("\nReaction at Node 1: {:.2f} W".format(F_reaction[0, 0]))
print("Reaction at Node 2: {:.2f} W".format(F_reaction[1, 0]))

# Problem 9.1
Construct row 1 of the B matrix for the six-node triangle.

**a.** Show that for rigid body translation, the $\epsilon_{xx}$ strain vanishes.

**b.** Let the nodal displacements be proportional to the coordinates, i.e., $u_x = ax$. Find the strain field. Does this answer make sense?


In [ ]:
'''
Compute row 1 of the B matrix
'''

def compute_B_row1():

    # Define symbolic variables for local coordinates
    xi, eta = sp.symbols('xi eta')

    # Define the quadratic shape functions for the six-node triangle
    N1 = xi * (2 * xi - 1)
    N2 = eta * (2 * eta - 1)
    N3 = (1 - xi - eta) * (2 * (1 - xi - eta) - 1)
    N4 = 4 * xi * eta
    N5 = 4 * eta * (1 - xi - eta)
    N6 = 4 * xi * (1 - xi - eta)

    # List of shape functions
    shape_functions = [N1, N2, N3, N4, N5, N6]

    # Assume nodal coordinates for simplicity
    nodal_coords = {
        'x1': 0, 'y1': 0,
        'x2': 1, 'y2': 0,
        'x3': 0, 'y3': 1,
        'x4': 0.5, 'y4': 0,
        'x5': 0.5, 'y5': 0.5,
        'x6': 0, 'y6': 0.5
    }

    # Map global coordinates (substitute numerical values for simplicity)
    x_global = sum(Ni * nodal_coords[f'x{i+1}'] for i, Ni in enumerate(shape_functions))
    y_global = sum(Ni * nodal_coords[f'y{i+1}'] for i, Ni in enumerate(shape_functions))

    # Compute the Jacobian matrix
    J = sp.Matrix([
        [sp.diff(x_global, xi), sp.diff(x_global, eta)],
        [sp.diff(y_global, xi), sp.diff(y_global, eta)]
    ])

    # Compute the Jacobian determinant and inverse
    J_det = J.det()
    if J_det != 0:
        J_inv = J.inv()

        # Transform derivatives of shape functions from (xi, eta) to (x, y)
        dN_dx = [J_inv[0, 0] * sp.diff(N, xi) + J_inv[0, 1] * sp.diff(N, eta) for N in shape_functions]
        dN_dy = [J_inv[1, 0] * sp.diff(N, xi) + J_inv[1, 1] * sp.diff(N, eta) for N in shape_functions]

        # Construct the first row of the B matrix
        B_row1 = []
        for i in range(len(dN_dx)):
            B_row1.append(dN_dx[i])
            B_row1.append(0)  # Zero placeholders for the uy components

        return sp.Matrix([B_row1])  # Return as a matrix for clean display
    else:
        raise ValueError("Jacobian is singular; check the mapping.")
B_row_1 = compute_B_row1()
print(f"The first row of the B matrix is:\n{B_row_1}")

In [ ]:
'''
Part a: Show that for rigid body translation, the strain is zero
'''

def verify_rigid_body_translation(B_row1):

    # Define the constant rigid body translation displacement field
    cx = sp.Symbol('c_x')  # Constant translation in x-direction
    cy = sp.Symbol('c_y')  # Constant translation in y-direction

    # Nodal displacement vector for rigid body translation
    nodal_displacements = [cx, cy] * (B_row1.shape[1] // 2)

    # Compute epsilon_xx using the first row of the B matrix and the displacement vector
    epsilon_xx = sum(B_row1[0, i] * nodal_displacements[i] for i in range(B_row1.shape[1]))

    # Simplify the result to check if epsilon_xx vanishes
    epsilon_xx_simplified = sp.simplify(epsilon_xx)

    print("Verification for rigid body translation:")
    print(f"ε_xx = {epsilon_xx_simplified}")


B_row1 = compute_B_row1()
print("Row 1 of the B-matrix (epsilon_xx):")
print(B_row1)



In [ ]:
'''
Part b.) let the nodal displacement be proportional to the nodal coordinates, ie. u_i = a * x_i, v_i = a * y_i
Find the stain field. Does this answer make sense?
'''

In [ ]:


def compute_B_row1():
    '''
    Computes the first row of the B-matrix for a six-node triangular finite element.
    This corresponds to ε_xx strain.
    '''
    # Define symbolic variables for local coordinates (natural coordinates)
    xi, eta = sp.symbols('xi eta')

    # Define the quadratic shape functions for the six-node triangle
    # These are derived based on the triangular element's geometry
    N1 = xi * (2 * xi - 1)
    N2 = eta * (2 * eta - 1)
    N3 = (1 - xi - eta) * (2 * (1 - xi - eta) - 1)
    N4 = 4 * xi * eta
    N5 = 4 * eta * (1 - xi - eta)
    N6 = 4 * xi * (1 - xi - eta)

    # List of shape functions
    shape_functions = [N1, N2, N3, N4, N5, N6]

    # Assume nodal coordinates for simplicity
    # These are example coordinates in the physical domain for the triangle
    nodal_coords = {
        'x1': 0, 'y1': 0,
        'x2': 1, 'y2': 0,
        'x3': 0, 'y3': 1,
        'x4': 0.5, 'y4': 0,
        'x5': 0.5, 'y5': 0.5,
        'x6': 0, 'y6': 0.5
    }

    # Map global coordinates (physical domain) using the shape functions
    x_global = sum(Ni * nodal_coords[f'x{i+1}'] for i, Ni in enumerate(shape_functions))
    y_global = sum(Ni * nodal_coords[f'y{i+1}'] for i, Ni in enumerate(shape_functions))

    # Compute the Jacobian matrix for mapping natural to global coordinates
    J = sp.Matrix([
        [sp.diff(x_global, xi), sp.diff(x_global, eta)],
        [sp.diff(y_global, xi), sp.diff(y_global, eta)]
    ])

    # Compute the Jacobian determinant and inverse
    J_det = J.det()
    if J_det != 0:
        # The Jacobian inverse transforms derivatives from natural to global coordinates
        J_inv = J.inv()

        # Transform derivatives of shape functions from (xi, eta) to (x, y)
        dN_dx = [J_inv[0, 0] * sp.diff(N, xi) + J_inv[0, 1] * sp.diff(N, eta) for N in shape_functions]
        dN_dy = [J_inv[1, 0] * sp.diff(N, xi) + J_inv[1, 1] * sp.diff(N, eta) for N in shape_functions]

        # Construct the first row of the B matrix
        # Row 1 corresponds to ε_xx = ∂u_x / ∂x
        B_row1 = []
        for i in range(len(dN_dx)):
            B_row1.append(dN_dx[i])
            B_row1.append(0)  # Zero placeholders for the u_y components

        return sp.Matrix([B_row1])  # Return as a matrix for clean display
    else:
        raise ValueError("Jacobian is singular; check the mapping.")

def compute_strain_field(B_row1):
    '''
    Computes the strain field (ε_xx) for the displacement field u_x = a*x and u_y = b*y.
    '''
    # Proportional displacements: u_x = a*x and u_y = b*y
    # These represent a uniform strain field induced by scaling in x and y directions.
    a, b = sp.symbols('a b')  # Constants of proportionality

    # Define nodal coordinates (physical domain for the triangle)
    nodal_coords = {
        'x1': 0, 'y1': 0,
        'x2': 1, 'y2': 0,
        'x3': 0, 'y3': 1,
        'x4': 0.5, 'y4': 0,
        'x5': 0.5, 'y5': 0.5,
        'x6': 0, 'y6': 0.5
    }

    # Compute nodal displacements proportional to coordinates
    # Each node's displacement depends linearly on its x and y coordinates
    nodal_displacements = []
    for i in range(1, 7):  # 6 nodes
        x_i = nodal_coords[f'x{i}']
        y_i = nodal_coords[f'y{i}']
        nodal_displacements.append(a * x_i)  # u_x = a * x
        nodal_displacements.append(b * y_i)  # u_y = b * y

    # Compute the strain field (ε_xx) using the first row of the B matrix
    epsilon_xx = sum(B_row1[0, i] * nodal_displacements[i] for i in range(B_row1.shape[1]))

    # Simplify the strain field
    epsilon_xx_simplified = sp.simplify(epsilon_xx)

    # Why it makes sense:
    # ε_xx = -1.0 * a corresponds to uniform strain along x-axis.
    # It depends only on a because u_x scales linearly with x.

    return epsilon_xx_simplified

def compute_additional_strains(B_matrix):
    '''
    Computes additional strain components (ε_yy and γ_xy) for the displacement field u_x = a*x, u_y = b*y.
    '''
    a, b = sp.symbols('a b')  # Constants for proportional displacement

    # Define nodal coordinates (as before)
    nodal_coords = {
        'x1': 0, 'y1': 0,
        'x2': 1, 'y2': 0,
        'x3': 0, 'y3': 1,
        'x4': 0.5, 'y4': 0,
        'x5': 0.5, 'y5': 0.5,
        'x6': 0, 'y6': 0.5
    }

    # Compute nodal displacements (as before)
    nodal_displacements = []
    for i in range(1, 7):  # 6 nodes
        x_i = nodal_coords[f'x{i}']
        y_i = nodal_coords[f'y{i}']
        nodal_displacements.append(a * x_i)  # u_x = a * x
        nodal_displacements.append(b * y_i)  # u_y = b * y

    # Compute epsilon_yy (Row 2 of B-matrix)
    epsilon_yy = sum(B_matrix[1, i] * nodal_displacements[i] for i in range(B_matrix.shape[1]))
    epsilon_yy_simplified = sp.simplify(epsilon_yy)

    # Compute gamma_xy (Row 3 of B-matrix)
    gamma_xy = sum(B_matrix[2, i] * nodal_displacements[i] for i in range(B_matrix.shape[1]))
    gamma_xy_simplified = sp.simplify(gamma_xy)

    # Why it makes sense:
    # ε_yy = 0 because there is no displacement change along y for u_y = b*y.
    # γ_xy = 0 because there is no shear displacement induced.

    return epsilon_yy_simplified, gamma_xy_simplified

# Main script
if __name__ == "__main__":
    # Compute Row 1 of the B-matrix
    B_row1 = compute_B_row1()
    #print("Row 1 of the B-matrix (ε_xx):")
    #print(B_row1)

    # Compute strain field ε_xx (Part b)
    epsilon_xx = compute_strain_field(B_row1)
    print("\nStrain field (ε_xx) for u_x = a*x, u_y = b*y:")
    print(epsilon_xx)

    # Extend B-matrix to include ε_yy and γ_xy (assume placeholder rows)
    B_full_matrix = sp.zeros(3, B_row1.shape[1])
    B_full_matrix[0, :] = B_row1  # Assign Row 1

    # Compute additional strain components
    epsilon_yy, gamma_xy = compute_additional_strains(B_full_matrix)
    print("\nStrain field (ε_yy) for u_x = a*x, u_y = b*y:")
    print(epsilon_yy)
    print("\nShear strain field (γ_xy) for u_x = a*x, u_y = b*y:")
    print(gamma_xy)






# Explanation of Strain Field Components

## 1. $\epsilon_{xx}$:
The strain $\epsilon_{xx}$ is defined as:
$$
\epsilon_{xx} = \frac{\partial u_x}{\partial x}
$$

### **Given Displacement Field**:
$$
u_x = a \cdot x
$$
Substituting into the strain definition:
$$
\epsilon_{xx} = \frac{\partial (a \cdot x)}{\partial x} = a
$$

The $B$-matrix introduces a factor of $-1.0$, resulting in:
$$
\epsilon_{xx} = -1.0 \cdot a
$$

### **Physical Meaning**:
- $\epsilon_{xx} = -1.0 \cdot a$ represents a **uniform compressive strain** in the $x$-direction if $a > 0$.
- The negative sign originates from the Jacobian transformation between natural ($\xi, \eta$) and global ($x, y$) coordinates.

---

## 2. $\epsilon_{yy}$:
The strain $\epsilon_{yy}$ is defined as:
$$
\epsilon_{yy} = \frac{\partial u_y}{\partial y}
$$

### **Given Displacement Field**:
$$
u_y = b \cdot y
$$
Substituting into the strain definition:
$$
\epsilon_{yy} = \frac{\partial (b \cdot y)}{\partial y} = b
$$

However, in this problem, the $B$-matrix yields:
$$
\epsilon_{yy} = 0
$$

### **Physical Meaning**:
- $\epsilon_{yy} = 0$ indicates no strain in the $y$-direction due to the given displacement field.

---

## 3. $\gamma_{xy}$:
The shear strain $\gamma_{xy}$ is defined as:
$$
\gamma_{xy} = \frac{\partial u_x}{\partial y} + \frac{\partial u_y}{\partial x}
$$

### **Given Displacement Field**:
$$
u_x = a \cdot x, \quad u_y = b \cdot y
$$

Substituting:
$$
\frac{\partial u_x}{\partial y} = 0, \quad \frac{\partial u_y}{\partial x} = 0
$$

Thus:
$$
\gamma_{xy} = 0 + 0 = 0
$$

### **Physical Meaning**:
- $\gamma_{xy} = 0$ indicates no shear strain in the element because the displacement field does not involve interactions between $x$- and $y$-directions.

---

## Summary Table:
| Strain Component  | Result             | Physical Meaning                                      |
|-------------------|--------------------|------------------------------------------------------|
| $\epsilon_{xx}$   | $-1.0 \cdot a$     | Uniform compressive strain in $x$-direction (if $a > 0$) |
| $\epsilon_{yy}$   | $0$                | No normal strain in $y$-direction                   |
| $\gamma_{xy}$     | $0$                | No shear strain between $x$- and $y$-directions     |



# Problem 9.3

Consider a quadrilateral domain model of unit thickness with a single finite element as shown in Figure 9.15. All dimensions are in meters. The traction applied on the edge 1–2 is normal to the edge and is given by $ 6 \cdot n N m^{-2}$, where n is the unit vector normal to the edge.
Calculate the element boundary force matrix.

![figure_5](figure_5.png)

In [ ]:
# Define symbols
xi, eta = sp.symbols('xi eta')  # xi (ξ) and eta (η) are the natural coordinates

In [ ]:
'''
Set up a matrix for the normal vector n for the edge 1-2
'''
# Edge vector from Node 1 to Node 2
edge_1_2 = sp.Matrix([-1.5, 0.6])  
# Normalize to get the unit vector
n_1_2 = edge_1_2 / edge_1_2.norm()  
print("Unit normal vector (n_1_2):")
sp.pprint(n_1_2)

In [ ]:
'''
Traction boundary condition
'''
# The traction on the edge is given as tau * n
tau = 6  # Magnitude of traction
t = tau * n_1_2  # Traction vector
print("\nTraction vector (t):")
sp.pprint(t)

In [ ]:
'''
Length of the edge 1-2
'''
length_1_2 = edge_1_2.norm()
print("\nLength of edge 1-2:")
sp.pprint(length_1_2)

In [ ]:
'''
Setting up the shape functions
'''
# Shape functions for the 4-node quadrilateral element
N_1 = (1/4) * (1-xi) * (1-eta)
N_2 = (1/4) * (1+xi) * (1-eta)
N_3 = (1/4) * (1+xi) * (1+eta)
N_4 = (1/4) * (1-xi) * (1+eta)

# Combine shape functions into a matrix
N = sp.Matrix([N_1, N_2, N_3, N_4])


In [ ]:
'''
Calculate the force contribution to each node
'''
# For edge 1-2, eta = -1, so shape functions simplify
#N_1_edge = (1/2) * (1 - xi)  # Shape function for Node 1 on edge 1-2
#N_2_edge = (1/2) * (1 + xi)  # Shape function for Node 2 on edge 1-2

# Force contributions to nodes on edge 1-2
f1 = sp.integrate(N_1 * t * (length_1_2 / 2), (xi, -1, 1))
f2 = sp.integrate(N_2 * t * (length_1_2 / 2), (xi, -1, 1))

# Combine results into the force vector for edge 1-2
f_edge = sp.Matrix([f1, f2, sp.S(0), sp.S(0)])  # Forces at Nodes 3 and 4 are 0
print("\nBoundary force vector (f_edge):")
sp.pprint(f_edge)




In [ ]:
'''
Substitute eta = -1 into the matrix for numerical results
'''
# This substitution is optional and should only happen later if needed
f_edge_eta_substituted = f_edge.subs(eta, -1)
print("\nBoundary force vector with eta = -1:")
sp.pprint(f_edge_eta_substituted)

# Problem 9.5

Consider a one-element triangular mesh shown in Figure 9.18. The boundary conditions are as follows. The edge BC is constrained in y and traction free in x, whereas the edge AB is constrained in x and traction free in y. The edge AC is subject to traction normal to the edge as shown in Figure 9.18. Assume Young’s modulus $ E = 3 \cdot 10^{11} $ Pa and Poisson’s ratio $ \nu = 0.3 $.

a. Construct the weak form corresponding to the generalized boundary conditions given in Section 9.5.

b. Construct the stiffness matrix.

c. Calculate the global force matrix.

d. Solve for the unknown displacement matrix and calculate the stress at (1.5,1.5).


In [ ]:
# Given values

E = 3E11
nu = 0.3
t = 15


In [ ]:
'''
Calculate the Coordinate Matrix
'''

XY = sp.Matrix([[0, 0], [3, 0], [0, 3]])

In [ ]:
'''
Plane stress constitutive matrix D
'''

D = (E / (1 - nu**2)) * np.array([
    [1, nu, 0],
    [nu, 1, 0],
    [0, 0, (1 - nu) / 2]
])

print("\nConstitutive matrix D:")
print(D)



In [ ]:
'''
Find the Shape Functions for 3-Node Triangular Element
'''
# Define natural coordinates (xi, eta)
xi, eta = sp.symbols('xi eta')

# Define shape functions for a three-node triangular element
N1 = 1 - xi - eta
N2 = xi
N3 = eta

# Collect shape functions into a matrix
N = sp.Matrix([N1, N2, N3])

print("\nShape Function Matrix N:")
print(N)

In [ ]:
'''
TEST
'''
# Define symbols
xi, eta = sp.symbols('xi eta')  # xi (ξ) and eta (η) are the natural coordinates

'''
Step 3: The Gradient
'''

# Calculate partial derivatives of shape functions with respect to xi and eta
dN_dxi = [sp.diff(Ni, xi) for Ni in N]
dN_deta = [sp.diff(Ni, eta) for Ni in N]

# Set the matrix for the gradient
GN = sp.Matrix([dN_dxi, dN_deta])  

print("\nGradient Matrix G:")
print(GN)

''' 
Jacobian Matrix
'''


# Jacobian Matrix J (2x3 gradient matrix multiplied by 3x2 coordinate matrix)
J = GN * XY


print("\nJacobian Matrix J:")
sp.pprint(J)






In [ ]:
'''
Determine the Jacobian Matrix, determinant, and the inverse
'''

# Derivatives of shape functions with respect to natural coordinates (xi, eta)
dN_dxi = sp.Matrix([sp.diff(N1, xi), sp.diff(N2, xi), sp.diff(N3, xi)])
dN_deta = sp.Matrix([sp.diff(N1, eta), sp.diff(N2, eta), sp.diff(N3, eta)])

# Set up the Jacobian matrix
# J = [x1 x2 x3] * [dN/dxi; dN/deta]
J = sp.Matrix([[dN_dxi.dot(XY[:, 0]), dN_dxi.dot(XY[:, 1])],
               [dN_deta.dot(XY[:, 0]), dN_deta.dot(XY[:, 1])]])

# Determinant of the Jacobian
det_J = J.det()

# Inverse of the Jacobian
J_inv = J.inv() if det_J != 0 else None

print("\nJacobian Matrix J:")
sp.pprint(J)

print("\nDeterminant of the Jacobian det(J):")
sp.pprint(det_J)

if J_inv:
    print("\nInverse of the Jacobian J_inv:")
    sp.pprint(J_inv)
else:
    print("\nThe Jacobian matrix is singular, no inverse exists.")

In [ ]:
'''
Strain-displacement matrix B
'''
# Derivatives of shape functions with respect to physical coordinates (x, y)
if J_inv:
    # Matrix containing derivatives of shape functions in natural coordinates
    dN_dxi_eta = sp.Matrix([[dN_dxi[0], dN_deta[0]],
                            [dN_dxi[1], dN_deta[1]],
                            [dN_dxi[2], dN_deta[2]]])
    
    # Multiply by the inverse Jacobian to get derivatives with respect to (x, y)
    dN_dx_dy = J_inv * dN_dxi_eta.T

    # Strain-displacement matrix B
    B = sp.zeros(3, 6)
    for i in range(3):
        dN_dx = dN_dx_dy[0, i]
        dN_dy = dN_dx_dy[1, i]

        # Fill the B matrix for each node
        B[0, 2 * i] = dN_dx     # dNi/dx term for normal strain in x
        B[1, 2 * i + 1] = dN_dy # dNi/dy term for normal strain in y
        B[2, 2 * i] = dN_dy     # dNi/dy in shear strain component
        B[2, 2 * i + 1] = dN_dx # dNi/dx in shear strain component

    print("\nStrain-Displacement Matrix B:")
    sp.pprint(B)
else:
    print("\nThe Jacobian matrix is singular, no inverse exists.")

In [ ]:
import sympy as sp

# Define symbols for natural coordinates
xi, eta = sp.symbols('xi eta')

# Integration points for 3-point quadrature rule in natural coordinates for a triangle
integration_points = [(0.1666666666, 0.1666666666),
                      (0.6666666666, 0.1666666666),
                      (0.1666666666, 0.6666666666)]

# Corresponding weights for 3-point quadrature for a triangle
weights = [0.1666666666, 0.1666666666, 0.1666666666]  # Weight of 1/6 for each point


# Initialize stiffness matrix (assuming a 3-node triangular element, resulting in a 6x6 matrix)
K = sp.zeros(6, 6)

# Loop over each integration point to accumulate the stiffness matrix
for (xi_i, eta_i), weight in zip(integration_points, weights):
    # Substitute xi_i and eta_i into the shape functions to calculate their derivatives
    dN_dxi = [sp.diff(Ni, xi) for Ni in N]
    dN_deta = [sp.diff(Ni, eta) for Ni in N]

    # Gradient matrix G with respect to natural coordinates (xi, eta)
    G = sp.Matrix([dN_dxi, dN_deta])

    # Calculate the Jacobian matrix J at the current integration point
    J = G * XY

    # Calculate the determinant of the Jacobian matrix
    det_J = J.det()
    if det_J == 0:
        raise ValueError("The Jacobian determinant is zero, the element might be degenerate.")

    # Calculate the inverse of the Jacobian matrix
    J_inv = J.inv()

    # Derivatives of shape functions with respect to physical coordinates (x, y)
    dN_dx_dy = J_inv * G

    # Strain-displacement matrix B
    B = sp.zeros(3, 6)
    for i in range(3):
        dN_dx = dN_dx_dy[0, i]
        dN_dy = dN_dx_dy[1, i]

        # Fill the B matrix for each node
        B[0, 2 * i] = dN_dx     # dNi/dx term for normal strain in x
        B[1, 2 * i + 1] = dN_dy # dNi/dy term for normal strain in y
        B[2, 2 * i] = dN_dy     # dNi/dy in shear strain component
        B[2, 2 * i + 1] = dN_dx # dNi/dx in shear strain component

    # Calculate the contribution to the stiffness matrix at the current point
    K_contribution = B.T * D * B * weight * abs(det_J)

    # Accumulate the contributions to form the element stiffness matrix
    K += K_contribution

# Display the final stiffness matrix
print("Stiffness Matrix K:")
sp.pprint(K)


In [ ]:
import sympy as sp

# Define your stiffness matrix K (this is a placeholder, replace it with your actual K)
K = sp.Matrix([
    [222527472438.462, 107142857100.0, -164835164769.231, -57692307669.2308, -57692307669.2308, -49450549430.7692],
    [107142857100.0, 222527472438.462, -49450549430.7692, -57692307669.2308, -57692307669.2308, -164835164769.231],
    [-164835164769.231, -49450549430.7692, 164835164769.231, 0, 0, 49450549430.7692],
    [-57692307669.2308, -57692307669.2308, 0, 57692307669.2308, 57692307669.2308, 0],
    [-57692307669.2308, -57692307669.2308, 0, 57692307669.2308, 57692307669.2308, 0],
    [-49450549430.7692, -164835164769.231, 49450549430.7692, 0, 0, 164835164769.231]
])

# Check if the matrix is symmetric
is_symmetric = K == K.T
print(f"Is the stiffness matrix symmetric? {is_symmetric}")

# Check the determinant to see if the matrix is full rank (det != 0 implies full rank)
det_K = K.det()
print(f"Determinant of K: {det_K}")

# Check the rank of the stiffness matrix
rank_K = K.rank()
print(f"Rank of K: {rank_K}")


In [ ]:
import sympy as sp

# Define traction applied on edge AC
t = 15  # Traction in N/m²

# Coordinates of nodes A and C (used to calculate length of the edge)
XA, YA = 0, 3
XC, YC = 3, 0

# Length of edge AC
L_AC = sp.sqrt((XC - XA)**2 + (YC - YA)**2)

# Shape functions for nodes A and C (assuming linear shape functions along the edge)
N_A = 1 - sp.Symbol('xi')  # Shape function for Node A
N_C = sp.Symbol('xi')      # Shape function for Node C

# Equivalent nodal forces for Nodes A and C due to traction t on edge AC
f_A = t * sp.integrate(N_A * L_AC / 2, ('xi', 0, 1))
f_C = t * sp.integrate(N_C * L_AC / 2, ('xi', 0, 1))

# Assemble the force vector (6x1) for the triangular element (force contributions for Nodes A, B, and C)
# Note: Only nodes A and C are affected by the traction, Node B has no force contribution
F = sp.Matrix([f_A, 0, 0, 0, f_C, 0])

# Display the force vector
print("Global Force Vector F:")
sp.pprint(F)


In [ ]:
# Define symbols for the displacements at nodes (u_Ax, u_Ay, u_Bx, u_By, u_Cx, u_Cy)
u_Ax, u_Ay, u_Bx, u_By, u_Cx, u_Cy = sp.symbols('u_Ax u_Ay u_Bx u_By u_Cx u_Cy')
u = sp.Matrix([u_Ax, u_Ay, u_Bx, u_By, u_Cx, u_Cy])

# Solve for the displacements
displacements = sp.linsolve((K, F), u)

# Display the displacements
print("Nodal Displacements:")
sp.pprint(displacements)
